In [177]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import utils.cleaning_dicts
#import matplotlib
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [178]:
%reload_ext autoreload
%autoreload 2

os.path.abspath(os.getcwd())
!ls

2022_w7.csv			       notebooks
chromedriver			       other_data
create_modeling_data_2022.ipynb        pfr
create_modeling_data_sample_all.ipynb  README.md
create_player_pools_2022.ipynb	       scripts
current_data			       spreads_data
fo_addtohist_update.csv		       sumconcrollrb.csv
historic_data			       update_spreads_file.py
misc_files			       utils
modeling_data			       weather_scraper_current_year.py


### IMPORTANT: Users must change the week values to the current week in the cell below

In [179]:
cur_week_int = 8
cur_week_str = str(8)

### Read in, clean and process all pff position datasets

In [180]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################

passing_depth = pd.read_csv('./historic_data/pff_data/passing_depth_hist.csv')
passing_allowed_pressure = pd.read_csv('./historic_data/pff_data/passing_allowed_pressure_hist.csv')
passing_pressure = pd.read_csv('./historic_data/pff_data/passing_pressure_hist.csv')
passing_concept = pd.read_csv('./historic_data/pff_data/passing_concept_hist.csv')
time_in_pocket = pd.read_csv('./historic_data/pff_data/time_in_pocket_hist.csv')
passing_summ_conc = pd.read_csv('./historic_data/pff_data/passing_summ_conc_hist.csv')

passing_depth_new = pd.read_csv('./scripts/nfl_all/passing_depth_2022.csv')
passing_allowed_pressure_new = pd.read_csv('./scripts/nfl_all/passing_allowed_pressure_2022.csv')
passing_pressure_new = pd.read_csv('./scripts/nfl_all/passing_pressure_2022.csv')
passing_concept_new = pd.read_csv('./scripts/nfl_all/passing_concept_2022.csv')
time_in_pocket_new = pd.read_csv('./scripts/nfl_all/time_in_pocket_2022.csv')
passing_summ_conc_new = pd.read_csv('./scripts/nfl_all/passing_summ_conc_2022.csv')
                                 
passing_depth = pd.concat([passing_depth, passing_depth_new], axis=0).reset_index(drop=True)
passing_allowed_pressure = pd.concat([passing_allowed_pressure, passing_allowed_pressure_new], axis=0).reset_index(drop=True)
passing_pressure = pd.concat([passing_pressure, passing_pressure_new], axis=0).reset_index(drop=True)
passing_concept = pd.concat([passing_concept, passing_concept_new], axis=0).reset_index(drop=True)
time_in_pocket = pd.concat([time_in_pocket, time_in_pocket_new], axis=0).reset_index(drop=True)
passing_summ_conc = pd.concat([passing_summ_conc, passing_summ_conc_new], axis=0).reset_index(drop=True)
                                 

def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df
    
passing_depth = drop_non_qbs(passing_depth)
passing_allowed_pressure = drop_non_qbs(passing_allowed_pressure)
passing_pressure = drop_non_qbs(passing_pressure)
passing_concept = drop_non_qbs(passing_concept)
time_in_pocket = drop_non_qbs(time_in_pocket)
passing_summ_conc = drop_non_qbs(passing_summ_conc)


passing_depth = passing_depth[passing_depth.columns.drop(list(passing_depth.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ### scripts/nfl_all
####################################################################################

rec_summ_conc = pd.read_csv('./historic_data/pff_data/rec_summ_conc_hist.csv')
receiving_concept = pd.read_csv('./historic_data/pff_data/receiving_concept_hist.csv')
receiving_depth = pd.read_csv('./historic_data/pff_data/receiving_depth_hist.csv')
receiving_scheme = pd.read_csv('./historic_data/pff_data/receiving_scheme_hist.csv')
                                 
rec_summ_conc_new = pd.read_csv('./scripts/nfl_all/rec_summ_conc_2022.csv')
receiving_concept_new = pd.read_csv('./scripts/nfl_all/receiving_concept_2022.csv')
receiving_depth_new = pd.read_csv('./scripts/nfl_all/receiving_depth_2022.csv')
receiving_scheme_new = pd.read_csv('./scripts/nfl_all/receiving_scheme_2022.csv')
                                 
rec_summ_conc = pd.concat([rec_summ_conc, rec_summ_conc_new], axis=0).reset_index(drop=True)
receiving_concept = pd.concat([receiving_concept, receiving_concept_new], axis=0).reset_index(drop=True)
receiving_depth = pd.concat([receiving_depth, receiving_depth_new], axis=0).reset_index(drop=True)
receiving_scheme = pd.concat([receiving_scheme, receiving_scheme_new], axis=0).reset_index(drop=True)                                 

def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./historic_data/pff_data/rush_summ_conc_hist.csv')
rush_summ_conc_new = pd.read_csv('./scripts/nfl_all/rush_summ_conc_2022.csv')                                 
                                 
rush_summ_conc = pd.concat([rush_summ_conc, rush_summ_conc_new], axis=0)
 

def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB|QB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################


block_summ_conc = pd.read_csv('./historic_data/pff_data/block_summ_conc_hist.csv')
offense_pass_blocking = pd.read_csv('./historic_data/pff_data/offense_pass_blocking_hist.csv')
offense_run_blocking = pd.read_csv('./historic_data/pff_data/offense_run_blocking_hist.csv')
                                 
block_summ_conc_new = pd.read_csv('./scripts/nfl_all/block_summ_conc_2022.csv')
offense_pass_blocking_new = pd.read_csv('./scripts/nfl_all/offense_pass_blocking_2022.csv')
offense_run_blocking_new = pd.read_csv('./scripts/nfl_all/offense_run_blocking_2022.csv')                                 

block_summ_conc = pd.concat([block_summ_conc, block_summ_conc_new], axis=0).reset_index(drop=True)
offense_pass_blocking = pd.concat([offense_pass_blocking, offense_pass_blocking_new], axis=0).reset_index(drop=True)
offense_run_blocking = pd.concat([offense_run_blocking, offense_run_blocking_new], axis=0).reset_index(drop=True)

def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

def_summ_conc = pd.read_csv('./historic_data/pff_data/def_summ_conc_hist.csv')
pass_rush_summary = pd.read_csv('./historic_data/pff_data/pass_rush_summary_hist.csv')
run_defense_summary = pd.read_csv('./historic_data/pff_data/run_defense_summary_hist.csv')
defense_coverage_scheme = pd.read_csv('./historic_data/pff_data/defense_coverage_scheme_hist.csv')
defense_coverage_summary = pd.read_csv('./historic_data/pff_data/defense_coverage_summary_hist.csv')
slot_coverage = pd.read_csv('./historic_data/pff_data/slot_coverage_hist.csv')
                                 
def_summ_conc_new = pd.read_csv('./scripts/nfl_all/def_summ_conc_2022.csv')
pass_rush_summary_new = pd.read_csv('./scripts/nfl_all/pass_rush_summary_2022.csv')
run_defense_summary_new = pd.read_csv('./scripts/nfl_all/run_defense_summary_2022.csv')
defense_coverage_scheme_new = pd.read_csv('./scripts/nfl_all/defense_coverage_scheme_2022.csv')
defense_coverage_summary_new = pd.read_csv('./scripts/nfl_all/defense_coverage_summary_2022.csv')
slot_coverage_new = pd.read_csv('./scripts/nfl_all/slot_coverage_2022.csv')

def_summ_conc = pd.concat([def_summ_conc, def_summ_conc_new], axis=0).reset_index(drop=True)
pass_rush_summary = pd.concat([pass_rush_summary, pass_rush_summary_new], axis=0).reset_index(drop=True)
run_defense_summary = pd.concat([run_defense_summary, run_defense_summary_new], axis=0).reset_index(drop=True)
defense_coverage_scheme = pd.concat([defense_coverage_scheme, defense_coverage_scheme_new], axis=0).reset_index(drop=True)
defense_coverage_summary = pd.concat([defense_coverage_summary, defense_coverage_summary_new], axis=0).reset_index(drop=True)
slot_coverage = pd.concat([slot_coverage, slot_coverage_new], axis=0).reset_index(drop=True)
                                 
def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)

def_summ_conc=def_summ_conc[def_summ_conc['position'].isin(["ed","lb","di","s","cb"])]
pass_rush_summary=pass_rush_summary[pass_rush_summary['position'].isin(["ed","lb","di","s"])]
run_defense_summary=run_defense_summary[run_defense_summary['position'].isin(["ed","lb","di","s","cb"])]
defense_coverage_scheme=defense_coverage_scheme[defense_coverage_scheme['position'].isin(["lb","s","cb"])]
defense_coverage_summary=defense_coverage_summary[defense_coverage_summary['position'].isin(["lb","s","cb"])]
slot_coverage=slot_coverage[slot_coverage['position'].isin(["lb","s","cb"])]

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	

st_kickers = pd.read_csv('./historic_data/pff_data/st_kickers_hist.csv')
st_punters = pd.read_csv('./historic_data/pff_data/st_punters_hist.csv')

st_kickers_new = pd.read_csv('./scripts/nfl_all/st_kickers_2022.csv')
st_punters_new = pd.read_csv('./scripts/nfl_all/st_punters_2022.csv')                                 
                                 
                                 
st_kickers = pd.concat([st_kickers, st_kickers_new], axis=0).reset_index(drop=True)
st_punters = pd.concat([st_punters, st_punters_new], axis=0).reset_index(drop=True)
                                 
def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

### Impute all missing values in pff dataframe - NEED TO UPDATE

In [181]:
%%time

def impute(df):
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reset_index(inplace=True, drop=True)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_id_impute'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
    return df

passing_depth = impute(passing_depth)
passing_allowed_pressure = impute(passing_allowed_pressure)
passing_pressure = impute(passing_pressure)
passing_concept = impute(passing_concept)
time_in_pocket = impute(time_in_pocket)
passing_summ_conc = impute(passing_summ_conc)

rec_summ_conc = impute(rec_summ_conc)
receiving_concept = impute(receiving_concept)
receiving_depth = impute(receiving_depth)
receiving_scheme = impute(receiving_scheme)

rush_summ_conc = impute(rush_summ_conc)

block_summ_conc = impute(block_summ_conc)
offense_pass_blocking = impute(offense_pass_blocking)
offense_run_blocking = impute(offense_run_blocking)

def_summ_conc = impute(def_summ_conc)
pass_rush_summary = impute(pass_rush_summary)
run_defense_summary = impute(run_defense_summary)
defense_coverage_scheme = impute(defense_coverage_scheme)
defense_coverage_summary = impute(defense_coverage_summary)
slot_coverage = impute(slot_coverage)

st_kickers = impute(st_kickers)
st_punters = impute(st_punters)

CPU times: user 1min 29s, sys: 251 ms, total: 1min 29s
Wall time: 1min 29s


### Add prefixes to all columns.  Creating column names structured as "source-dataset_column-name"

In [182]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id','player_team_id','unique_team_id','team_id_impute','player','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','player','team_id_impute','numeric_id','position','team_name','unique_team_id','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc = id_prefix(prefix="pass_summary_", df=passing_summ_conc)
rush_summ_conc = id_prefix(prefix="rush_summary_", df=rush_summ_conc)
rec_summ_conc = id_prefix(prefix="rec_summary_", df=rec_summ_conc)
block_summ_conc = id_prefix(prefix="block_summary_", df=block_summ_conc)
def_summ_conc = id_prefix(prefix="def_summary_", df=def_summ_conc)
st_kickers = id_prefix(prefix="kicking_", df=st_kickers)
st_punters = id_prefix(prefix="punting_", df=st_punters)


passing_depth = create_prefix(prefix="pass_depth_", df=passing_depth)
passing_allowed_pressure = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure)
passing_pressure = create_prefix(prefix="pass_under_pressure_", df=passing_pressure)
passing_concept = create_prefix(prefix="pass_concept_", df=passing_concept)
time_in_pocket = create_prefix(prefix="pass_time_", df=time_in_pocket)


receiving_concept = create_prefix(prefix="rec_concept_", df=receiving_concept)
receiving_depth = create_prefix(prefix="rec_depth_", df=receiving_depth)
receiving_scheme = create_prefix(prefix="rec_scheme_", df=receiving_scheme)

offense_pass_blocking = create_prefix(prefix="pass_block_", df=offense_pass_blocking)
offense_run_blocking = create_prefix(prefix="run_block_", df=offense_run_blocking)


pass_rush_summary = create_prefix(prefix="pass_rush_", df=pass_rush_summary)
run_defense_summary = create_prefix(prefix="run_defense_", df=run_defense_summary)
defense_coverage_scheme = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme)
defense_coverage_summary = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary)
slot_coverage = create_prefix(prefix="def_slot_coverage_", df=slot_coverage)



### Read in weather data and clean raiders name - merged onto spreads data below ###

In [183]:
### read in weather data###
weather = pd.read_csv('./current_data/week_'+cur_week_str+'/weather_hist_all.csv')

def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2021' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2022' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)

### Create spreads data ###

In [184]:
####################################################################################
				###   spreads data cleaning and engineering ###
####################################################################################

spreads = pd.read_csv('./current_data/week_'+cur_week_str+'/spreadsw'+cur_week_str+'.csv')

new_acc = {'oak':'lv',
          'sd':'lac',
          'stl':'lar'}  

spreads['team_home_abb'] = spreads['team_home_abb'].map(new_acc).fillna(spreads['team_home_abb'])
spreads['away_team_abb'] = spreads['away_team_abb'].map(new_acc).fillna(spreads['away_team_abb']) 

spreads = spreads[spreads['schedule_season']>=2014]
spreads = spreads[['schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line','starting_spread', 'Total Score Open',
       'fav_team_open', 'fav_team_cur', 'remain_fav', 'spread_movement','ou_movement', 'strong_movement', 'fav_team_stronger']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)


def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)



spreads['schedule_season'] = spreads['schedule_season'].apply(int)    
spreads['schedule_week'] = spreads['schedule_week'].apply(int)  
data = spreads.sort_values(by=["team_home_abb","schedule_season","schedule_week"], ascending=[True, True, True])

def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)  
    df['team_home_abb'] = df['team_home_abb'].map(new_acc).fillna(df['team_home_abb'])
    df['away_team_abb'] = df['away_team_abb'].map(new_acc).fillna(df['away_team_abb'])
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
data = clean_spreads(data)

data = pd.merge(data, weather, on='away_matchup_id', how='left')


sh = data
sa = data

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora1(nData):
    if nData['team_favorite_id'] == nData['team_home_abb']:
        return 1
    elif nData['team_notfav_id'] == nData['team_home_abb']:
        return 1
    else:
        return 0
spread_comb['homeoraway'] = spread_comb.apply(lambda nData: hora1(nData), axis=1)

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)
#sh['fav_homeoraway'] = sh.apply(lambda nData: hora(nData), axis=1)

def ws(nData):
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0

def ls(nData):    
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0

spread_comb['ats_w'] = spread_comb.apply(lambda nData: ws(nData), axis=1)
spread_comb['ats_l'] = spread_comb.apply(lambda nData: ls(nData), axis=1)

## Create Football Outsiders rolling function

In [185]:


def rolling_fo(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = ['total_dvoa', 'off_dvoa','off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa','def_rush_dvoa', 'special_teams_dvoa']
    ids = data[['team_id', 'year', 'team', 'week', 'opp']].reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df

## Read in historic weekly football outsiders data and create the current week rows for each team

In [186]:
	##Create the current weeks fo team_ids/rows to roll into##
fo_data = pd.read_csv("./current_data/week_"+cur_week_str+"/fo_weekly_update.csv")
fo_data_new = fo_data[~fo_data['week'].isnull()]
fo_data_new=fo_data_new.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)
fo_data_new['team_id']=fo_data_new['team_id'].str.replace("2022_"+str(cur_week_int-1), str("2022_"+cur_week_str))

fo_data_new = fo_data_new.sort_values(by=["team","week"], ascending=[True, False])
fo_data_new[fo_data_new.columns[4:]] = np.nan
fo_data_new.tail().T

,218,219,220,221,222
team_id,sea_2022_8,sf_2022_8,tb_2022_8,ten_2022_8,was_2022_8
year,2022,2022,2022,2022,2022
team,sea,sf,tb,ten,was
week,8,8,8,8,8
opp,NaN,NaN,NaN,NaN,NaN
total_dvoa,NaN,NaN,NaN,NaN,NaN
off_dvoa,NaN,NaN,NaN,NaN,NaN
off_pass_dvoa,NaN,NaN,NaN,NaN,NaN
off_rush_dvoa,NaN,NaN,NaN,NaN,NaN
def_dvoa,NaN,NaN,NaN,NaN,NaN


## Now read in the historic FO data and concat all of them together for our rolling function

In [188]:
fo_data_2022 = pd.read_csv("./historic_data/fo_data/fo_weekly_hist.csv")
fo_data = pd.read_csv("./current_data/week_"+cur_week_str+"/fo_weekly_update.csv")

fo = pd.concat([fo_data_2022, fo_data, fo_data_new], axis=0).reset_index(drop=True)

fo['team'] = fo['team'].map(new_acc).fillna(fo['team'])
fo['opp'] = fo['opp'].map(new_acc).fillna(fo['opp']) 

fo['team'] = fo['team'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['team'])
fo['opp'] = fo['opp'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['opp'])

##combine our current season fo data with the new week 4 rows we just made##
fo_roll = rolling_fo(data=fo, roll_value=3, roll_type='mean')
fo_roll = fo_roll.rename(columns={'team_id': 'unique_team_id'})

fo_roll['unique_team_id']=fo_roll['unique_team_id'].str.replace('sd_','lac_')
fo_roll['unique_team_id']=fo_roll['unique_team_id'].str.replace('oak_','lv_')
#fo_roll.drop(['year','team','week','opp'], axis=1, inplace=True)

fo_roll.head().T

,0,1,2,3,4
unique_team_id,ari_2014_1,ari_2014_2,ari_2014_3,ari_2014_5,ari_2014_6
year,2014,2014,2014,2014,2014
team,ari,ari,ari,ari,ari
week,1,2,3,5,6
opp,lac,nyg,sf,den,was
total_dvoa,NaN,0.59,0.625,0.613333,0.56
off_dvoa,NaN,0.53,0.505,0.55,0.533333
off_pass_dvoa,NaN,0.53,0.45,0.516667,0.486667
off_rush_dvoa,NaN,0.51,0.57,0.55,0.56
def_dvoa,NaN,29.8,21.65,10.433333,-2.8


### PFF team_game_summaries (tgs) clean and create current week rows

In [190]:
tgs_new_week = pd.read_csv("./current_data/week_"+cur_week_str+"/team_game_summaries_w"+cur_week_str+".csv")

tgs_new_week = tgs_new_week[~tgs_new_week['week'].isnull()]
tgs_new_week=tgs_new_week.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)

tgs_new_week['team_name'] = tgs_new_week['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs_new_week['team'])
tgs_new_week['opponent_name'] = tgs_new_week['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs_new_week['opponent'])

tgs_new_week['home_or_away']=tgs_new_week['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs_new_week['home_team'] = tgs_new_week.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs_new_week['away_team'] = tgs_new_week.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)
    
    df['team_name'] = df['team_name'].map(new_acc).fillna(df['team_name'])
    df['opponent_name'] = df['opponent_name'].map(new_acc).fillna(df['opponent_name'])


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    
    return df

tgs_new_week = clean_pff_team_summ(tgs_new_week)
tgs_new_week['wl_int'] = ''
tgs_new_week = tgs_new_week.sort_values(by=["team_name","week"], ascending=[True, False])

## Now read in historic tgs data and clean

In [191]:
tgs_data_2022 = pd.read_csv("./historic_data/pff_data/team_game_summaries_historic.csv")
tgs_data_cur = pd.read_csv("./current_data/week_"+cur_week_str+"/team_game_summaries_w"+cur_week_str+".csv")
tgs = pd.concat([tgs_data_2022, tgs_data_cur], axis=0)

tgs = tgs[tgs['year'] >= 2014]


tgs['team_name'] = tgs['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

##adding just incase accronyms have changed
tgs['team_name'] = tgs['team_name'].map(new_acc).fillna(tgs['team_name'])
tgs['opponent_name'] = tgs['opponent_name'].map(new_acc).fillna(tgs['opponent_name']) 

tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)
    
    df['team_name'] = df['team_name'].map(new_acc).fillna(df['team_name'])
    df['opponent_name'] = df['opponent_name'].map(new_acc).fillna(df['opponent_name'])


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    
    ##Impute missing special teams data added after 2014##
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reset_index(inplace=True, drop=True)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_name'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
   
    return df


   
tgs_clean = clean_pff_team_summ(tgs)


tgs_clean = pd.concat([tgs_clean, tgs_new_week], axis=0).reset_index(drop=True)
tgs_clean['year']=tgs_clean['year'].apply(int)
tgs_clean['week']=tgs_clean['week'].apply(int)
tgs_clean['special_teams']=tgs_clean['special_teams'].apply(float)
tgs_clean = tgs_clean.sort_values(by=["team_name","year","week"], ascending=[True, True, True])

tgs_clean = tgs_clean[['unique_team_id','team_id_impute', 'home_matchup_id','opponent_id','wl','pf','pa','team_name','opponent_name','year','week','overall_performance', 'offense', 'pass',
       'pass_blocking', 'receiving', 'rushing', 'run_blocking', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage', 'special_teams']]

tgs_clean.drop_duplicates(inplace=True)

In [192]:
tgs_clean.tail()

,unique_team_id,team_id_impute,home_matchup_id,opponent_id,wl,pf,pa,team_name,opponent_name,year,...,pass_blocking,receiving,rushing,run_blocking,defense,rush_defense,tackling,pass_rush,coverage,special_teams
4340,was_2022_4,was_2022,dalvswas_2022_4,dal_2022_4,L,10,25,was,dal,2022,...,60.3,63.1,70.3,72.9,71.9,72.0,68.8,66.8,68.1,79.9
4341,was_2022_5,was_2022,wasvsten_2022_5,ten_2022_5,L,17,21,was,ten,2022,...,52.9,67.8,61.4,42.9,67.0,64.3,70.7,74.2,64.6,43.9
4342,was_2022_6,was_2022,chivswas_2022_6,chi_2022_6,W,12,7,was,chi,2022,...,75.6,49.9,60.4,72.1,72.1,48.4,39.6,88.6,71.9,84.9
4343,was_2022_7,was_2022,wasvsgb_2022_7,gb_2022_7,W,23,21,was,gb,2022,...,42.1,71.7,65.6,50.0,59.0,71.9,55.0,53.8,59.0,66.9
4375,was_2022_8,was_2022,wasvsgb_2022_8,gb_2022_8,W,23,21,was,gb,2022,...,42.1,71.7,65.6,50.0,59.0,71.9,55.0,53.8,59.0,66.9


### Create tgs rolling mean function and combine all tgs datasets together and pass through the rolling function

In [193]:
def rolling_tgs(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team_name","year","week"], ascending=[True, True, True])
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
        
tgs_roll = rolling_tgs(data=tgs_clean, roll_value=3, roll_type='mean')

tgs_roll = tgs_roll[['unique_team_id','wl','pf','pa','overall_performance', 'offense', 'pass',
       'pass_blocking', 'receiving', 'rushing', 'run_blocking', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage', 'special_teams']]

tgs_roll = tgs_roll.rename(columns={c: c+'_tgs' for c in tgs_roll.columns if c not in ['unique_team_id','wl','pf','pa']})

tgs_roll.rename(columns={'unique_team_id_tgs_pff':'unique_team_id'}, inplace=True)

## Read in all the pff current week datasets and prep for rolling function

In [194]:
passing_depth_new = pd.read_csv("./current_data/week_"+cur_week_str+"/passing_depth_new_pp_w"+cur_week_str+".csv")
passing_allowed_pressure_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_allowed_pressure_new_pp_w'+cur_week_str+".csv")
passing_pressure_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_pressure_new_pp_w'+cur_week_str+".csv")
passing_concept_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_concept_new_pp_w'+cur_week_str+".csv")
time_in_pocket_new = pd.read_csv('./current_data/week_'+cur_week_str+'/time_in_pocket_new_pp_w'+cur_week_str+".csv")
passing_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_summ_conc_new_pp_w'+cur_week_str+".csv")


rec_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/rec_summ_conc_pp_w'+cur_week_str+".csv")
receiving_concept_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_concept_pp_w'+cur_week_str+".csv")
receiving_depth_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_depth_pp_w'+cur_week_str+".csv")
receiving_scheme_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_scheme_pp_w'+cur_week_str+".csv")

rush_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/rush_summ_conc_pp_w'+cur_week_str+".csv")

block_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/block_summ_conc_pp_w'+cur_week_str+".csv")
offense_pass_blocking_new = pd.read_csv('./current_data/week_'+cur_week_str+'/offense_pass_blocking_pp_w'+cur_week_str+".csv")
offense_run_blocking_new = pd.read_csv('./current_data/week_'+cur_week_str+'/offense_run_blocking_pp_w'+cur_week_str+".csv")

def_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/def_summ_conc_pp_w'+cur_week_str+".csv")
pass_rush_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/pass_rush_summary_pp_w'+cur_week_str+".csv")
run_defense_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/run_defense_summary_pp_w'+cur_week_str+".csv")
defense_coverage_scheme_new = pd.read_csv('./current_data/week_'+cur_week_str+'/defense_coverage_scheme_pp_w'+cur_week_str+".csv")
defense_coverage_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/defense_coverage_summary_pp_w'+cur_week_str+".csv")
slot_coverage_new = pd.read_csv('./current_data/week_'+cur_week_str+'/slot_coverage_pp_w'+cur_week_str+".csv")

st_kickers_new = pd.read_csv('./current_data/week_'+cur_week_str+'/st_kickers_pp_w'+cur_week_str+".csv")
st_punters_new = pd.read_csv('./current_data/week_'+cur_week_str+'/st_punters_no_inj_pp_w'+cur_week_str+".csv")


passing_depth_new['week'] = cur_week_str 
passing_allowed_pressure_new['week'] = cur_week_str 
passing_pressure_new['week'] = cur_week_str 
passing_concept_new['week'] = cur_week_str 
time_in_pocket_new['week'] = cur_week_str 
passing_summ_conc_new['week'] = cur_week_str 
rec_summ_conc_new['week'] = cur_week_str 
receiving_concept_new['week'] = cur_week_str
receiving_depth_new['week'] = cur_week_str 
receiving_scheme_new['week'] = cur_week_str 
rush_summ_conc_new['week'] = cur_week_str
block_summ_conc_new['week'] = cur_week_str 
offense_pass_blocking_new['week'] = cur_week_str 
offense_run_blocking_new['week'] = cur_week_str 
def_summ_conc_new['week'] = cur_week_str 
pass_rush_summary_new['week'] = cur_week_str 
run_defense_summary_new['week'] = cur_week_str 
defense_coverage_scheme_new['week'] = cur_week_str 
defense_coverage_summary_new['week'] = cur_week_str 
slot_coverage_new['week'] = cur_week_str 
st_kickers_new['week'] = cur_week_str 
st_punters_new['week'] = cur_week_str



In [195]:
def_summ_conc_new.tail()

,p_id,unique_team_id,player_team_id,team_id_impute,player,numeric_id,position,team_name,player_game_count,assists,...,tackles,targets,total_pressures,touchdowns,yards,yards_after_catch,yards_per_reception,week,year,plyr_number
647,perrionwinfrey_cle_2022_8,cle_2022_8,perrionwinfrey_cle_2022,cle_2022,perrionwinfrey,122952,di,cle,1,0,...,0,0,0,0,0,0,0,8,2022,8
648,jaylenwatson_kc_2022_8,kc_2022_8,jaylenwatson_kc_2022,kc_2022,jaylenwatson,131960,cb,kc,1,3,...,0,0,0,0,0,0,0,8,2022,8
649,samroberts_ne_2022_8,ne_2022_8,samroberts_ne_2022,ne_2022,samroberts,156069,di,ne,1,0,...,0,0,0,0,0,0,0,8,2022,8
650,joshuawilliams_kc_2022_8,kc_2022_8,joshuawilliams_kc_2022,kc_2022,joshuawilliams,156083,cb,kc,1,1,...,0,0,0,0,0,0,0,8,2022,8
651,christianmatthew_ari_2022_8,ari_2022_8,christianmatthew_ari_2022,ari_2022,christianmatthew,156140,cb,ari,1,0,...,0,0,0,0,0,0,0,8,2022,8


### Add the prefixes like we did for the pff datasets above

In [196]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id','player_team_id','unique_team_id','team_id_impute','player','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','plyr_number','player','team_id_impute','numeric_id','position','team_name','unique_team_id','numeric_id','position','team_name','year','week','plyr_number'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','plyr_number','team_id_impute','numeric_id','position','team_name','year','week','plyr_number'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc_new = id_prefix(prefix="pass_summary_", df=passing_summ_conc_new)
rush_summ_conc_new  = id_prefix(prefix="rush_summary_", df=rush_summ_conc_new)
rec_summ_conc_new  = id_prefix(prefix="rec_summary_", df=rec_summ_conc_new)
block_summ_conc_new  = id_prefix(prefix="block_summary_", df=block_summ_conc_new)
def_summ_conc_new  = id_prefix(prefix="def_summary_", df=def_summ_conc_new)
st_kickers_new  = id_prefix(prefix="kicking_", df=st_kickers_new)
st_punters_new  = id_prefix(prefix="punting_", df=st_punters_new)


passing_depth_new = create_prefix(prefix="pass_depth_", df=passing_depth_new)
passing_allowed_pressure_new = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure_new)
passing_pressure_new = create_prefix(prefix="pass_under_pressure_", df=passing_pressure_new)
passing_concept_new = create_prefix(prefix="pass_concept_", df=passing_concept_new)
time_in_pocket_new = create_prefix(prefix="pass_time_", df=time_in_pocket_new)


receiving_concept_new = create_prefix(prefix="rec_concept_", df=receiving_concept_new)
receiving_depth_new = create_prefix(prefix="rec_depth_", df=receiving_depth_new)
receiving_scheme_new = create_prefix(prefix="rec_scheme_", df=receiving_scheme_new)

offense_pass_blocking_new = create_prefix(prefix="pass_block_", df=offense_pass_blocking_new)
offense_run_blocking_new = create_prefix(prefix="run_block_", df=offense_run_blocking_new)


pass_rush_summary_new = create_prefix(prefix="pass_rush_", df=pass_rush_summary_new)
run_defense_summary_new = create_prefix(prefix="run_defense_", df=run_defense_summary_new)
defense_coverage_scheme_new = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme_new)
defense_coverage_summary_new = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary_new)
slot_coverage_new= create_prefix(prefix="def_slot_coverage_", df=slot_coverage_new)


In [197]:
def_summ_conc_new.tail()

,p_id,player,player_team_id,unique_team_id,team_id_impute,numeric_id,position,team_name,year,week,...,def_summary_snap_counts_run_defense,def_summary_snap_counts_slot,def_summary_stops,def_summary_tackles,def_summary_targets,def_summary_total_pressures,def_summary_touchdowns,def_summary_yards,def_summary_yards_after_catch,def_summary_yards_per_reception
647,perrionwinfrey_cle_2022_8,perrionwinfrey,perrionwinfrey_cle_2022,cle_2022_8,cle_2022,122952,di,cle,2022,8,...,0,0,0,0,0,0,0,0,0,0
648,jaylenwatson_kc_2022_8,jaylenwatson,jaylenwatson_kc_2022,kc_2022_8,kc_2022,131960,cb,kc,2022,8,...,0,0,0,0,0,0,0,0,0,0
649,samroberts_ne_2022_8,samroberts,samroberts_ne_2022,ne_2022_8,ne_2022,156069,di,ne,2022,8,...,0,0,0,0,0,0,0,0,0,0
650,joshuawilliams_kc_2022_8,joshuawilliams,joshuawilliams_kc_2022,kc_2022_8,kc_2022,156083,cb,kc,2022,8,...,0,0,0,0,0,0,0,0,0,0
651,christianmatthew_ari_2022_8,christianmatthew,christianmatthew_ari_2022,ari_2022_8,ari_2022,156140,cb,ari,2022,8,...,0,0,0,0,0,0,0,0,0,0


### Bring the historic and new player pool data together

In [198]:
passing_depth = pd.concat([passing_depth, passing_depth_new], axis=0)
passing_allowed_pressure = pd.concat([passing_allowed_pressure, passing_allowed_pressure_new], axis=0)
passing_pressure = pd.concat([passing_pressure, passing_pressure_new], axis=0)
passing_concept = pd.concat([passing_concept, passing_concept_new], axis=0)
time_in_pocket = pd.concat([time_in_pocket, time_in_pocket_new], axis=0)
passing_summ_conc = pd.concat([passing_summ_conc, passing_summ_conc_new], axis=0)


rec_summ_conc = pd.concat([rec_summ_conc, rec_summ_conc_new], axis=0)
receiving_concept = pd.concat([receiving_concept, receiving_concept_new], axis=0)
receiving_depth = pd.concat([receiving_depth, receiving_depth_new], axis=0)
receiving_scheme = pd.concat([receiving_scheme, receiving_scheme_new], axis=0)

rush_summ_conc = pd.concat([rush_summ_conc, rush_summ_conc_new], axis=0)

block_summ_conc = pd.concat([block_summ_conc, block_summ_conc_new], axis=0)
offense_pass_blocking = pd.concat([offense_pass_blocking, offense_pass_blocking_new], axis=0)
offense_run_blocking = pd.concat([offense_run_blocking, offense_run_blocking_new], axis=0)

def_summ_conc = pd.concat([def_summ_conc, def_summ_conc_new], axis=0)
pass_rush_summary = pd.concat([pass_rush_summary, pass_rush_summary_new], axis=0)
run_defense_summary = pd.concat([run_defense_summary, run_defense_summary_new], axis=0)
defense_coverage_scheme = pd.concat([defense_coverage_scheme, defense_coverage_scheme_new], axis=0)
defense_coverage_summary = pd.concat([defense_coverage_summary, defense_coverage_summary_new], axis=0)
slot_coverage = pd.concat([slot_coverage, slot_coverage_new], axis=0)

st_kickers = pd.concat([st_kickers, st_kickers_new], axis=0)
st_punters = pd.concat([st_punters, st_punters_new], axis=0)


### after the concat cell ###
passing_depth.drop_duplicates(subset='p_id', inplace=True)
passing_allowed_pressure.drop_duplicates(subset='p_id', inplace=True)
passing_pressure.drop_duplicates(subset='p_id', inplace=True)
passing_concept.drop_duplicates(subset='p_id', inplace=True)
time_in_pocket.drop_duplicates(subset='p_id', inplace=True)
passing_summ_conc.drop_duplicates(subset='p_id', inplace=True)


rec_summ_conc.drop_duplicates(subset='p_id', inplace=True)
receiving_concept.drop_duplicates(subset='p_id', inplace=True)
receiving_depth.drop_duplicates(subset='p_id', inplace=True)
receiving_scheme.drop_duplicates(subset='p_id', inplace=True)

rush_summ_conc.drop_duplicates(subset='p_id', inplace=True)

block_summ_conc.drop_duplicates(subset='p_id', inplace=True)
offense_pass_blocking.drop_duplicates(subset='p_id', inplace=True)
offense_run_blocking.drop_duplicates(subset='p_id', inplace=True)

def_summ_conc.drop_duplicates(subset='p_id', inplace=True)
pass_rush_summary.drop_duplicates(subset='p_id', inplace=True)
run_defense_summary.drop_duplicates(subset='p_id', inplace=True)
defense_coverage_scheme.drop_duplicates(subset='p_id', inplace=True)
defense_coverage_summary.drop_duplicates(subset='p_id', inplace=True)
slot_coverage.drop_duplicates(subset='p_id', inplace=True)

st_kickers.drop_duplicates(subset='p_id', inplace=True)
st_punters.drop_duplicates(subset='p_id', inplace=True)


## Create rolling function and pass pff datasets through

In [220]:
%%time

def rolling(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["player","team_name","year","week"], ascending=[True, True, True, True])
    data['week']=data['week'].apply(str)
    data['year']=data['year'].apply(str)
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
   
passing_depth_roll = rolling(data=passing_depth, roll_value=3, roll_type='mean')
passing_allowed_pressure_roll = rolling(data=passing_allowed_pressure, roll_value=3, roll_type='mean')
passing_pressure_roll = rolling(data=passing_pressure, roll_value=3, roll_type='mean')
passing_concept_roll = rolling(data=passing_concept, roll_value=3, roll_type='mean')
time_in_pocket_roll = rolling(data=time_in_pocket, roll_value=3, roll_type='mean')
passing_summ_conc_roll = rolling(data=passing_summ_conc, roll_value=3, roll_type='mean')


rec_summ_conc_roll = rolling(data=rec_summ_conc, roll_value=3, roll_type='mean')
receiving_concept_roll =rolling(data=receiving_concept, roll_value=3, roll_type='mean')
receiving_depth_roll = rolling(data=receiving_depth, roll_value=3, roll_type='mean')
receiving_scheme_roll = rolling(data=receiving_scheme, roll_value=3, roll_type='mean')

rush_summ_conc_roll = rolling(data=rush_summ_conc, roll_value=3, roll_type='mean')

block_summ_conc_roll = rolling(data=block_summ_conc, roll_value=3, roll_type='mean')
offense_pass_blocking_roll = rolling(data=offense_pass_blocking, roll_value=3, roll_type='mean')
offense_run_blocking_roll = rolling(data=offense_run_blocking, roll_value=3, roll_type='mean')

def_summ_conc_roll = rolling(data=def_summ_conc, roll_value=3, roll_type='mean')
pass_rush_summary_roll = rolling(data=pass_rush_summary, roll_value=3, roll_type='mean')
run_defense_summary_roll = rolling(data=run_defense_summary, roll_value=3, roll_type='mean')
defense_coverage_scheme_roll = rolling(data=defense_coverage_scheme, roll_value=3, roll_type='mean')
defense_coverage_summary_roll = rolling(data=defense_coverage_summary, roll_value=3, roll_type='mean')
slot_coverage_roll = rolling(data=slot_coverage, roll_value=3, roll_type='mean')

st_kickers_roll = rolling(data=st_kickers, roll_value=3, roll_type='mean')
st_punters_roll = rolling(data=st_punters, roll_value=3, roll_type='mean')

CPU times: user 8min 20s, sys: 422 ms, total: 8min 20s
Wall time: 8min 20s


In [221]:
rush_summ_conc_roll = rolling(data=rush_summ_conc, roll_value=3, roll_type='mean')


## TO DO: Create better imputation function before weighting team_position_group functions

def filter_fillna(df=None, position=None, min_Var=None):
    sub= df[df['position'].str.match(position)]
    sub_limit = sub[(sub[min_Var] <=5) & (sub[min_Var] >=1)]
    buckup_df = pd.DataFrame(sub_limit.median()).T
    num_cols = sub.select_dtypes(include=[np.number]).columns.tolist()
    msk = sub.isnull()
    tmp = sub[num_cols].mask(msk, buckup_df[num_cols])
    tmp = np.where(msk[num_cols], buckup_df[num_cols], tmp[num_cols])
    tmp = pd.DataFrame(tmp, columns=buckup_df.columns)
    ids = pd.DataFrame(sub.select_dtypes(exclude=[np.number])).reset_index(drop=True)
    mrg = pd.concat([ids, tmp], axis=1)
    return mrg
    

In [222]:
%%time

def impute(df):
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reset_index(inplace=True, drop=True)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_id_impute'])[num_cols].fillna(df.median()).reset_index(level=0, drop=True)
    return df

passing_depth_roll = impute(passing_depth_roll)
passing_allowed_pressure_roll = impute(passing_allowed_pressure_roll)
passing_pressure_roll = impute(passing_pressure_roll)
passing_concept_roll = impute(passing_concept_roll)
time_in_pocket_roll = impute(time_in_pocket_roll)
passing_summ_conc_roll = impute(passing_summ_conc_roll)

rec_summ_conc_roll = impute(rec_summ_conc_roll)
receiving_concept_roll = impute(receiving_concept_roll)
receiving_depth_roll = impute(receiving_depth_roll)
receiving_scheme_roll = impute(receiving_scheme_roll)

rush_summ_conc_roll = impute(rush_summ_conc_roll)

block_summ_conc_roll = impute(block_summ_conc_roll)
offense_pass_blocking_roll = impute(offense_pass_blocking_roll)
offense_run_blocking_roll = impute(offense_run_blocking_roll)

def_summ_conc_roll = impute(def_summ_conc_roll)
pass_rush_summary_roll = impute(pass_rush_summary_roll)
run_defense_summary_roll = impute(run_defense_summary_roll)
defense_coverage_scheme_roll = impute(defense_coverage_scheme_roll)
defense_coverage_summary_roll = impute(defense_coverage_summary_roll)
slot_coverage_roll = impute(slot_coverage_roll)

st_kickers_roll = impute(st_kickers_roll)
st_punters_roll = impute(st_punters_roll)

CPU times: user 51.1 s, sys: 184 ms, total: 51.2 s
Wall time: 51.2 s


In [223]:
rush_summ_conc_roll = impute(rush_summ_conc_roll)

In [224]:
biometrics = pd.read_csv('./other_data/2022_imputed_combine.csv'); biometrics.columns

biometrics=biometrics[['unique_id','positionclean','height_clean',
       'weight_clean', 'speed_clean', 'hand_size', 'arm_length', 'bench',
       'vertical', 'broad_jump', 'shuttle', '3cone', 'explosive', 'size_speed',
       'draft_yr', 'round', 'selection']]

biometrics['positionclean']=biometrics['positionclean'].apply(str)

qb_bio = biometrics[biometrics['positionclean'].isin(['qb'])]
rb_bio = biometrics[biometrics['positionclean'].isin(['rb','qb','fb','wr'])]
rec_bio = biometrics[biometrics['positionclean'].isin(['wr','te','rb'])]
ol_bio = biometrics[biometrics['positionclean'].isin(['ol','te'])]
def_bio = biometrics[biometrics['positionclean'].isin(['dl','db','lb'])]
st_bio = biometrics[biometrics['positionclean'].isin(['st'])]
qb_bio.head()

,unique_id,positionclean,height_clean,weight_clean,speed_clean,hand_size,arm_length,bench,vertical,broad_jump,shuttle,3cone,explosive,size_speed,draft_yr,round,selection
17487,aaronbrooks_oak_2006,qb,75.5,203,4.59,9.50,32.00,24.000001,35.5,118.0,4.29,7.52,20.795279,0.250141,1999,4,131
17488,aaronbrooks_oak_2007,qb,75.5,203,4.59,9.50,32.00,24.000000,35.5,118.0,4.29,7.52,20.795279,0.250141,1999,4,131
17489,aaronrodgers_gb_2006,qb,74.0,223,4.75,10.13,32.25,23.999998,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24
17490,aaronrodgers_gb_2007,qb,74.0,223,4.75,10.13,32.25,24.000000,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24
17491,aaronrodgers_gb_2008,qb,74.0,223,4.75,10.13,32.25,24.000000,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24


In [225]:
rb_bio.head()

,unique_id,positionclean,height_clean,weight_clean,speed_clean,hand_size,arm_length,bench,vertical,broad_jump,shuttle,3cone,explosive,size_speed,draft_yr,round,selection
17487,aaronbrooks_oak_2006,qb,75.5,203,4.59,9.50,32.00,24.000001,35.5,118.0,4.29,7.52,20.795279,0.250141,1999,4,131
17488,aaronbrooks_oak_2007,qb,75.5,203,4.59,9.50,32.00,24.000000,35.5,118.0,4.29,7.52,20.795279,0.250141,1999,4,131
17489,aaronrodgers_gb_2006,qb,74.0,223,4.75,10.13,32.25,23.999998,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24
17490,aaronrodgers_gb_2007,qb,74.0,223,4.75,10.13,32.25,24.000000,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24
17491,aaronrodgers_gb_2008,qb,74.0,223,4.75,10.13,32.25,24.000000,34.5,110.0,4.32,7.38,21.887624,0.262899,2005,1,24


In [226]:
imp = rb_bio.groupby(['positionclean']).median().reset_index()
imp = imp.rename(columns={'positionclean': 'position'})

imp['position'] = imp['position'].str.replace('rb','hb')
rush_summ_conc_roll['position'] = rush_summ_conc_roll['position'].str.replace('fb','hb')

temp = pd.merge(rush_summ_conc_roll, imp, on='position', how='left')

rush_summ_conc_roll_mrg = pd.merge(rush_summ_conc_roll, rb_bio, left_on='player_team_id', right_on='unique_id', how='left')
rush_summ_conc_roll_mrg.drop(['unique_id','positionclean'], axis=1, inplace=True)

#rush_summ_conc_roll_mrg[rush_summ_conc_roll_mrg.isnull()] = temp
#df_test = rush_summ_conc_roll_mrg.combine_first(temp)

In [227]:
rush_summ_conc_roll_mrg.columns

Index(['p_id', 'player', 'player_team_id', 'unique_team_id', 'team_id_impute',
       'position', 'team_name', 'year', 'week', 'numeric_id',
       'rush_summary_player_game_count', 'rush_summary_attempts',
       'rush_summary_avoided_tackles', 'rush_summary_breakaway_attempts',
       'rush_summary_breakaway_percent', 'rush_summary_breakaway_yards',
       'rush_summary_declined_penalties', 'rush_summary_designed_yards',
       'rush_summary_drops', 'rush_summary_elu_recv_mtf',
       'rush_summary_elu_rush_mtf', 'rush_summary_elu_yco',
       'rush_summary_elusive_rating', 'rush_summary_explosive',
       'rush_summary_first_downs', 'rush_summary_franchise_id',
       'rush_summary_fumbles', 'rush_summary_gap_attempts',
       'rush_summary_grades_hands_fumble', 'rush_summary_grades_offense',
       'rush_summary_grades_offense_penalty', 'rush_summary_grades_pass',
       'rush_summary_grades_pass_block', 'rush_summary_grades_pass_route',
       'rush_summary_grades_run', 'rush_summ

In [ ]:
df_mrg[df_mrg.isnull()] = temp

df_test = df_mrg.combine_first(temp)

In [103]:
rb_bio.columns = [str(col) + '_rbs' for col in rb_bio.columns]
rush_summ_conc_roll = pd.merge(rush_summ_conc_roll, rb_bio, left_on='player_team_id', right_on='unique_id_rbs', how='left')
rush_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
rush_summ_conc_roll.to_csv('sumconcrollrb.csv', index=False)

# Combine players for each dataset into team_year_week groupings

### These next few cells will compute weighted averages based on average snaps played.  The first function will default snaps to 1 if snap value is 0.  The rest of the functions are dataset specific and will compute the weighted averages based on rollup aaverages and snaps played.

#### For example: Washington had 5 rbs player in the last 3 games.  It doesn't make sense to weight all the players stats into a single average if 3 of those backs only averaged 2 snaps and rushed for 2 yards whereas B. Robinson averages 18 snaps and rushes for 65 yards and Gibson averages 10 snaps for 40 yards.  Therefore we weight each players rolling average based on their rolling snaps played. 

## Compute rushing weighted average dataset

In [228]:
rb_bio.columns = [str(col) + '_rbs' for col in rb_bio.columns]
rush_summ_conc_roll = pd.merge(rush_summ_conc_roll, rb_bio, left_on='player_team_id', right_on='unique_id_rbs', how='left')
rush_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
rush_summ_conc_roll = impute(rush_summ_conc_roll)

## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
def rush_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

rush_summ_conc_roll['rush_summary_attempts'] = rush_summ_conc_roll.apply(lambda df: rush_att(df, var='rush_summary_attempts'), axis=1)   


def weighted(nData, snap_Var='rush_summary_attempts'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)


rb_stats = rush_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()
rb_stats.tail(n=10)


,unique_team_id,numeric_id,rush_summary_attempts,rush_summary_avoided_tackles,rush_summary_breakaway_attempts,rush_summary_breakaway_percent,rush_summary_breakaway_yards,rush_summary_declined_penalties,rush_summary_designed_yards,rush_summary_drops,...,bench_rbs,vertical_rbs,broad_jump_rbs,shuttle_rbs,3cone_rbs,explosive_rbs,size_speed_rbs,draft_yr_rbs,round_rbs,selection_rbs
4366,was_2021_7,54944.238806,11.159204,0.452736,0.437811,13.309453,6.786070,0.0,41.808458,0.218905,...,16.641965,35.029851,118.835821,4.252987,7.122550,24.227904,0.247183,2018.283582,3.626866,90.761194
4367,was_2021_8,52388.646341,9.552846,0.711382,0.357724,10.874797,5.544715,0.0,34.796748,0.235772,...,16.837635,35.237805,119.585366,4.253782,7.115254,24.204689,0.246086,2018.268293,3.426829,85.573171
4368,was_2022_1,9443.000000,4.000000,0.666667,0.000000,0.000000,0.000000,0.0,16.333333,0.000000,...,18.045870,34.532510,117.630686,4.270644,7.125022,22.147680,0.246470,2018.200000,3.400000,84.800000
4369,was_2022_2,44382.178571,8.785714,2.345238,0.000000,0.000000,0.000000,0.0,34.404762,0.000000,...,17.321429,35.070988,118.516439,4.255744,7.118268,23.934558,0.246118,2018.571429,3.392857,81.928571
4370,was_2022_3,48362.180000,9.180000,2.286667,0.050000,5.000000,1.050000,0.0,29.553333,0.560000,...,16.560000,34.669506,118.038411,4.280034,7.129060,24.022610,0.249851,2018.340000,3.940000,102.040000
4371,was_2022_4,59740.018182,10.296970,1.806061,0.000000,0.000000,0.000000,0.0,31.715152,0.593939,...,16.272727,34.562963,117.261470,4.275135,7.165351,24.199711,0.249042,2018.909091,4.145455,105.709091
4372,was_2022_5,59058.777778,10.333333,1.185185,0.000000,0.000000,0.000000,0.0,31.314815,0.481481,...,17.111111,34.638889,118.222222,4.268377,7.139800,24.246625,0.249603,2018.666667,2.888889,65.777778
4373,was_2022_6,54118.573333,7.675556,0.213333,0.062222,5.133333,2.053333,0.0,23.346667,0.288889,...,17.293333,34.583676,117.548641,4.264289,7.115773,23.360079,0.251793,2017.253333,4.226667,112.920000
4374,was_2022_7,45355.021505,8.383513,0.922939,0.435484,23.215233,8.025090,0.0,30.706093,0.215054,...,17.677419,34.780787,117.817146,4.274745,7.096207,23.145824,0.251805,2016.440860,3.892473,104.698925
4375,was_2022_8,49254.759563,9.731330,1.209472,0.644809,31.087796,12.863388,0.0,37.641166,0.130237,...,17.781421,34.951236,118.275276,4.269028,7.087835,23.212661,0.250474,2016.497268,3.743169,100.098361


## Compute Passing weight average datasets

In [229]:
qb_bio.columns = [str(col) + '_qbs' for col in qb_bio.columns]
passing_summ_conc_roll = pd.merge(passing_summ_conc_roll, qb_bio, left_on='player_team_id', right_on='unique_id_qbs', how='left')
passing_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
passing_summ_conc_roll = impute(passing_summ_conc_roll)

def pass_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]
passing_summ_conc_roll['pass_summary_dropbacks'] = passing_summ_conc_roll.apply(lambda df: pass_att(df, var='pass_summary_dropbacks'), axis=1)
passing_depth_roll['pass_depth_base_dropbacks'] = passing_depth_roll.apply(lambda df: pass_att(df, var='pass_depth_base_dropbacks'), axis=1)  
passing_pressure_roll['pass_under_pressure_base_dropbacks'] = passing_pressure_roll.apply(lambda df: pass_att(df, var='pass_under_pressure_base_dropbacks'), axis=1)  
passing_allowed_pressure_roll['pressure_source_allowed_pressure_dropbacks'] = passing_allowed_pressure_roll.apply(lambda df: pass_att(df, var='pressure_source_allowed_pressure_dropbacks'), axis=1)  
#passing_concept_roll['pass_concept_dropbacks'] = passing_summ_conc_roll.apply(lambda df: pass_att(df, var='pass_concept_dropbacks'), axis=1)  
time_in_pocket_roll['pass_time_dropbacks'] = time_in_pocket_roll.apply(lambda df: pass_att(df, var='pass_time_dropbacks'), axis=1)     


def weighted(nData, snap_Var='pass_summary_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
qb_stats = passing_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_depth_base_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
pass_depth_stats = passing_depth_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pressure_source_allowed_pressure_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_allowed_pressure_stats = passing_allowed_pressure_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_under_pressure_base_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_pressure_stats = passing_pressure_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_concept_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_concept_stats = passing_concept_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_time_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
time_in_pocke_stats = time_in_pocket_roll.groupby('unique_team_id').apply(weighted).reset_index()

## Compute receiver weighted average datasets

In [230]:
rec_bio.columns = [str(col) + '_wrs' for col in rec_bio.columns]
rec_summ_conc_roll = pd.merge(rec_summ_conc_roll, rec_bio, left_on='player_team_id', right_on='unique_id_wrs', how='left')
rec_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
rec_summ_conc_roll = impute(rec_summ_conc_roll)


## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
def rec_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

rec_summ_conc_roll['rec_summary_targets'] = rec_summ_conc_roll.apply(lambda df: rec_att(df, var='rec_summary_targets'), axis=1)   
receiving_concept_roll['rec_concept_base_targets'] = receiving_concept_roll.apply(lambda df: rec_att(df, var='rec_concept_base_targets'), axis=1) 
receiving_depth_roll['rec_depth_base_targets'] = receiving_depth_roll.apply(lambda df: rec_att(df, var='rec_depth_base_targets'), axis=1) 
receiving_scheme_roll['rec_scheme_base_targets'] = receiving_scheme_roll.apply(lambda df: rec_att(df, var='rec_scheme_base_targets'), axis=1) 


def weighted(nData, snap_Var='rec_summary_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)

rec_stats = rec_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_concept_base_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_concept = receiving_concept.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_depth_base_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_depth = receiving_depth.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_scheme_base_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_scheme = receiving_scheme.groupby('unique_team_id').apply(weighted).reset_index()

## Compute OL weighted average dataset

In [231]:
ol_bio.columns = [str(col) + '_ols' for col in ol_bio.columns]
block_summ_conc_roll = pd.merge(block_summ_conc_roll, ol_bio, left_on='player_team_id', right_on='unique_id_ols', how='left')
block_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
block_summ_conc_roll = impute(block_summ_conc_roll)

def snap_fix(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

block_summ_conc_roll['block_summary_snap_counts_offense'] = block_summ_conc_roll.apply(lambda df: snap_fix(df, var='block_summary_snap_counts_offense'), axis=1)
offense_pass_blocking_roll['pass_block_snap_counts_pass_block'] = offense_pass_blocking_roll.apply(lambda df: snap_fix(df, var='pass_block_snap_counts_pass_block'), axis=1) 
offense_run_blocking_roll['run_block_snap_counts_run_block'] = offense_run_blocking_roll.apply(lambda df: snap_fix(df, var='run_block_snap_counts_run_block'), axis=1) 


def weighted(nData, snap_Var='block_summary_snap_counts_offense'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
ol_stats = block_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_block_snap_counts_pass_block'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
offense_pass_blocking_roll = offense_pass_blocking_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='run_block_snap_counts_run_block'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
offense_run_blocking_roll = offense_run_blocking_roll.groupby('unique_team_id').apply(weighted).reset_index()



## Compute defensive weighted averages datasets

In [232]:
def_bio.columns = [str(col) + '_defs' for col in def_bio.columns]
def_summ_conc_roll = pd.merge(def_summ_conc_roll, def_bio, left_on='player_team_id', right_on='unique_id_defs', how='left')
def_summ_conc_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)
def_summ_conc_roll = impute(def_summ_conc_roll)


def snap_fixs(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

def_summ_conc_roll['def_summary_snap_counts_defense'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_summary_snap_counts_defense'), axis=1) 
def_summ_conc_roll['def_summary_snap_counts_run_defense'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_summary_snap_counts_run_defense'), axis=1) 
def_summ_conc_roll['def_summary_snap_counts_pass_rush'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_summary_snap_counts_pass_rush'), axis=1) 
def_summ_conc_roll['def_summary_snap_counts_coverage'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_summary_snap_counts_coverage'), axis=1) 


# pass_rush_summary_roll['pass_rush_snap_counts_pass_play'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='pass_rush_snap_counts_pass_play'), axis=1)
# run_defense_summary_roll['run_defense_snap_counts_run'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='run_defense_snap_counts_run'), axis=1)
# defense_coverage_scheme_roll['def_coverage_scheme_base_snap_counts_coverage'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_coverage_scheme_base_snap_counts_coverage'), axis=1)
# defense_coverage_summary_roll['def_coverage_summary_coverage_snaps_per_target'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_coverage_summary_coverage_snaps_per_target'), axis=1)
# slot_coverage_roll['def_slot_coverage_snap_counts_defense'] = def_summ_conc_roll.apply(lambda df: snap_fixs(df, var='def_slot_coverage_snap_counts_defense'), axis=1)




## Subset into defense positional groups ##
def_rundef = def_summ_conc_roll[def_summ_conc_roll['position'].isin(['ed','di','lb'])]
def_passrush = def_summ_conc_roll[def_summ_conc_roll['position'].isin(['lb','ed','di'])]
def_cov = def_summ_conc_roll[def_summ_conc_roll['position'].isin(['lb','cb','s'])]


def weighted(nData, snap_Var='def_summary_snap_counts_defense'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
def_stats = def_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='def_summary_snap_counts_run_defense'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
def_rundef = def_rundef.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='def_summary_snap_counts_pass_rush'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
def_passrush = def_passrush.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='def_summary_snap_counts_coverage'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
def_cov = def_cov.groupby('unique_team_id').apply(weighted).reset_index()





# def weighted(nData, snap_Var='pass_rush_snap_counts_pass_play'):
#     data_cols = nData.select_dtypes(include=[np.number])
#     num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
#     return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
# pass_rush_stats = pass_rush_summary_roll.groupby('unique_team_id').apply(weighted).reset_index()

# def weighted(nData, snap_Var='run_defense_snap_counts_run'):
#     data_cols = nData.select_dtypes(include=[np.number])
#     num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
#     return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
# run_defense_stats = run_defense_summary_roll.groupby('unique_team_id').apply(weighted).reset_index()

# def weighted(nData, snap_Var='def_coverage_summary_coverage_snaps_per_target'):
#     data_cols = nData.select_dtypes(include=[np.number])
#     num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
#     return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
# defense_coverage_summary_stats = defense_coverage_summary_roll.groupby('unique_team_id').apply(weighted).reset_index()

# def weighted(nData, snap_Var='def_coverage_scheme_base_snap_counts_coverage'):
#     data_cols = nData.select_dtypes(include=[np.number])
#     num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
#     return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
# defense_coverage_scheme_stats = defense_coverage_scheme_roll.groupby('unique_team_id').apply(weighted).reset_index()

# def weighted(nData, snap_Var='def_slot_coverage_snap_counts_defense'):
#     data_cols = nData.select_dtypes(include=[np.number])
#     num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
#     return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
# slot_coverage_stats = slot_coverage_roll.groupby('unique_team_id').apply(weighted).reset_index()

#def_stats = pd.merge(def_stats, def_rundef, on='unique_team_id', how='inner').merge(def_passrush, on='unique_team_id', how='inner').merge(def_cov, on='unique_team_id', how='inner')


def_stats.head()



,unique_team_id,numeric_id,def_summary_assists,def_summary_batted_passes,def_summary_catch_rate,def_summary_declined_penalties,def_summary_forced_fumbles,def_summary_franchise_id,def_summary_grades_coverage_defense,def_summary_grades_defense,...,bench_defs,vertical_defs,broad_jump_defs,shuttle_defs,3cone_defs,explosive_defs,size_speed_defs,draft_yr_defs,round_defs,selection_defs
0,ari_2014_1,9074.000000,0.333333,0.000000,70.226821,0.000000,0.000000,16.000000,60.584477,61.850000,...,21.210526,32.776271,115.778812,4.411314,7.242630,22.841669,0.282246,2010.157895,3.631579,104.315789
1,ari_2014_10,5453.149401,0.380565,0.021261,73.237718,0.040382,0.012700,1.000000,63.770461,64.293564,...,18.858733,33.144971,116.685233,4.331833,7.161912,22.533098,0.275249,2009.447774,3.556507,101.102312
2,ari_2014_11,5259.055969,0.356216,0.000000,70.635314,0.022154,0.019968,1.000000,63.691524,64.336686,...,18.245300,33.443009,117.542952,4.302931,7.106221,22.560918,0.270711,2009.178400,3.360297,94.331876
3,ari_2014_12,5376.940635,0.350820,0.000000,69.261585,0.000000,0.030531,1.857766,63.717993,66.110585,...,18.394475,33.400880,117.210515,4.320045,7.123781,22.624919,0.272260,2009.342622,3.527502,99.279864
4,ari_2014_13,5505.065511,0.474310,0.056840,72.629271,0.000000,0.029383,1.852601,62.197780,65.108462,...,19.007707,33.473970,117.331735,4.312366,7.117551,22.672571,0.271194,2009.492775,3.810694,108.049133


## Compute special teams weighted averages

In [233]:
st_bio.columns = [str(col) + '_st' for col in st_bio.columns]
st_kickers_roll = pd.merge(st_kickers_roll, st_bio, left_on='player_team_id', right_on='unique_id_st', how='left')
st_kickers_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)

def kicks_fix(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]
		
st_kickers_roll['kicks'] = st_kickers_roll['kicking_pat_attempts']+st_kickers_roll['kicking_total_attempts']
st_kickers_roll ['kicks'] = st_kickers_roll .apply(lambda df: snap_fixs(df, var='kicks'), axis=1)

def weighted(nData, snap_Var='kicks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
st_kickers = st_kickers_roll.groupby('unique_team_id').apply(weighted).reset_index()

In [234]:
st_punters_roll = pd.merge(st_punters_roll, st_bio, left_on='player_team_id', right_on='unique_id_st', how='left')
st_punters_roll.drop_duplicates(subset=['p_id'], keep='first', inplace=True)

def punts_fix(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]
		
st_punters_roll['punting_attempts'] = st_punters_roll.apply(lambda df: snap_fixs(df, var='punting_attempts'), axis=1)

def weighted(nData, snap_Var='punting_attempts'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
st_punters = st_punters_roll.groupby('unique_team_id').apply(weighted).reset_index()

## Create Modeling File and write out to modeling_data directory

In [235]:
spread_vars = spread_comb[spread_comb['schedule_week'] != '1']



from functools import reduce


spread_ids = spread_vars[['team_id','home_matchup_id']]
spread_ids.columns = ['unique_team_id','home_matchup_id']

spread_targs = spread_vars[['team_id',
'schedule_week',
'schedule_season',
'team_favorite_id',
'spread_favorite',
'over_under_line',
'fav_cover',
'over_under_result',
'fav_homeoraway',
'remain_fav',
'spread_movement',
"ou_movement",
"strong_movement",
"fav_team_stronger",
"temperature",
"wind_mph",
"dome",
"precip"]]


dfs_list = [spread_ids,
            tgs_roll,
            fo_roll,
            qb_stats,
            rb_stats,
            rec_stats,
            ol_stats,
           def_stats,
           def_rundef,
           def_cov,
           def_passrush,
           st_punters,
           st_kickers]

dfs_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'],
                                            how='left'), dfs_list)

def fav_ids(nData):
    if str(nData['team_favorite_id']) in str(nData['team_id']):
        return nData['team_id']
    else:
        pass
spread_targs['fav_team_id'] = spread_targs.apply(lambda nData: fav_ids(nData), axis=1)


favs = spread_targs[~spread_targs['fav_team_id'].isnull()]
not_fav = spread_targs[spread_targs['fav_team_id'].isnull()]

not_fav_df = dfs_team[dfs_team.unique_team_id.isin(not_fav.team_id)]

dfs_team = dfs_team.rename(columns={c: c+'_fav' for c in dfs_team.columns if c not in ['unique_team_id','team_id','schedule_week','schedule_season','home_matchup_id','spread_favorite','over_under_line','fav_cover','over_under_result','wl','pf','pa']})
not_fav_df = not_fav_df.rename(columns={c: c+'_dog' for c in not_fav_df.columns if c not in ['unique_team_id','team_id','schedule_week','schedule_season','home_matchup_id','spread_favorite','over_under_line','fav_cover','over_under_result','wl','pf','pa']})

favs = favs[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]
not_fav = not_fav[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]

## Merge files and write to modeling_data directory

In [236]:
fin_df = pd.merge(favs, dfs_team, left_on='team_id', right_on='unique_team_id', how='left').merge(not_fav_df, on='home_matchup_id', how='left')
fin_df=fin_df.round(2)
fin_df.drop_duplicates(subset='home_matchup_id',inplace=True)
fin_df.to_csv('./modeling_data/nfl_spreads_w'+cur_week_str+'.csv', index=False)

### UNDER CONSTRUCTION: Creating function to create modeling file by user selected datasets

In [ ]:
from functools import reduce


spread_ids = spread_vars[['team_id','home_matchup_id']]
spread_ids.columns = ['unique_team_id','home_matchup_id']

spread_targs = spread_vars[['team_id',
'schedule_week',
'schedule_season',
'team_favorite_id',
'spread_favorite',
'over_under_line',
'fav_cover',
'over_under_result',
'fav_homeoraway',
'remain_fav',
'spread_movement',
"ou_movement",
"strong_movement",
"fav_team_stronger",
"temperature",
"wind_mph",
"dome",
"precip"]]


sample=[spread_vars,tgs_clean,fo_roll,qb_stats,rb_stats,rec_stats,ol_stats,def_stats,def_rundef,def_cov,def_passrush,st_punters,st_kickers]
           
def fav_ids(nData):
    if str(nData['team_favorite_id']) in str(nData['team_id']):
        return nData['team_id']
    else:
        pass
        
def build_model_dataset(data_list=None):
    """
        Args:
        data_list: User provides a list of dataframes in format - [df1, df2, df3...] to be used to create modeling dataset.
        
        Options: 
        Football Outsiders
        fo_roll - 
        
        PFF
        -Team Game Summaries -
        tgs_roll -

        -Passing:
        qb_stats -
        passing_depth_stats -
        passing_pressure_stats -
        passing_allowed_pressure_stats -
        passing_concept_stats -
        time_in_pocket_stats -
        
        -Receiving:
        rec_stats -
        receiving_concept -
        receiving_depth -
        receiving_scheme -
        
        -Blocking:
        ol_stats -
        offense_pass_blocking_roll -
        offense_run_blocking_roll -
        
        -Defense:
        def_stats -
        def_rundef -
        def_passrush -
        def_cov -
        pass_rush_stats -
        defense_coverage_summary_stats -
        run_defense_stats -
        defense_coverage_scheme_stats -
        slot_coverage_stats -
        
        -Special Teams:
        st_kickers -
        st_punters - 
    """
    
    dataset_list = [spread_ids]+data_list
    dfs_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'], how='left'), dataset_list)
    spread_targs['fav_team_id'] = spread_targs.apply(lambda nData: fav_ids(nData), axis=1)
    favs = spread_targs[~spread_targs['fav_team_id'].isnull()]
    not_fav = spread_targs[spread_targs['fav_team_id'].isnull()]

    not_fav_df = dfs_team[dfs_team.unique_team_id.isin(not_fav.team_id)]

    favs = favs[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]
    not_fav = not_fav[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]
    return pd.merge(favs, dfs_team, left_on='team_id', right_on='unique_team_id', how='left').merge(not_fav_df, on='home_matchup_id', how='left')

fin_df=build_model_dataset(data_list=sample)
fin_df=fin_df.round(2)
fin_df.drop_duplicates(subset='home_matchup_id',inplace=True)
#fin_df.to_csv('./modeling_data/nfl_spreads_w'+cur_week_str+'.csv', index=False)

In [ ]:
fin_df=fin_df.round(2)
fin_df.drop_duplicates(subset='home_matchup_id',inplace=True)

In [ ]:
fin_df.to_csv('./modeling_data/nfl_spreads_w'+cur_week_str+'.csv', index=False)